# Day la Giao dien (GUI) de xu ly file Phase Space from DBSCAN

In [19]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog, Button, Label
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows


def analyze_files():
    # Hiển thị hộp thoại chọn thư mục
    folder_path = filedialog.askdirectory()
    if not folder_path:
        return

    # Tạo danh sách để lưu trữ kết quả
    results = []

    # Tạo một workbook mới để lưu dữ liệu và hình ảnh
    wb = Workbook()
    ws_summary = wb.active
    ws_summary.title = "Summary"

    # Duyệt qua tất cả các tệp trong thư mục
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.phsp'):  # Chỉ xử lý các tệp có đuôi .phsp
            file_path = os.path.join(folder_path, file_name)

            # Đọc tệp vào DataFrame
            columns = ['Event_number', 'Single_strand_breaks', 'Double_strand_breaks',
                       'Complex_strand_breaks', 'Cluster_sizes', 'Cluster_size_weights']
            data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns)

            # Tính tổng số strand breaks cho tệp hiện tại
            sum_single_strand_breaks = data['Single_strand_breaks'].sum()
            sum_double_strand_breaks = data['Double_strand_breaks'].sum()
            sum_complex_strand_breaks = data['Complex_strand_breaks'].sum()

            # Lưu kết quả vào một từ điển
            file_result = {
                'File Name': file_name,
                'Single Strand Breaks': sum_single_strand_breaks,
                'Double Strand Breaks': sum_double_strand_breaks,
                'Complex Strand Breaks': sum_complex_strand_breaks
            }

            # Thêm từ điển vào danh sách kết quả
            results.append(file_result)

            # Vẽ biểu đồ cho từng tệp
            plt.figure(figsize=(8, 6))
            plt.bar(['Single Strand Breaks', 'Double Strand Breaks', 'Complex Strand Breaks'],
                    [sum_single_strand_breaks, sum_double_strand_breaks, sum_complex_strand_breaks])
            plt.xlabel('Strand Break Type')
            plt.ylabel('Sum')
            plt.title(f'Sum of Strand Breaks for {file_name}')
            plt.grid(axis='y', linestyle='--', alpha=0.7)

            # Lưu biểu đồ thành hình ảnh
            chart_path = os.path.join(folder_path, f"{file_name}.png")
            plt.savefig(chart_path)
            plt.close()

            # Thêm một sheet mới vào workbook cho mỗi file
            ws = wb.create_sheet(title=file_name[:30])  # Tên sheet phải ngắn hơn 31 ký tự
            img = Image(chart_path)
            ws.add_image(img, 'A1')

    # Chuyển đổi kết quả thành DataFrame và ghi vào trang tổng hợp
    results_df = pd.DataFrame(results)
    for r in dataframe_to_rows(results_df, index=False, header=True):
        ws_summary.append(r)

    # Hiển thị hộp thoại để chọn đường dẫn lưu tệp
    output_path = filedialog.asksaveasfilename(defaultextension=".xlsx",
                                               filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])

    # Kiểm tra nếu người dùng đã chọn một đường dẫn
    if output_path:
        # Lưu workbook vào file Excel
        wb.save(output_path)

        # Thông báo thành công
        print(results_df)
        print(f"Excel file saved to: {output_path}")
        label.config(text=f"Result saved to: {output_path}")
    else:
        label.config(text="Save operation was canceled.")


# Tạo cửa sổ chính của Tkinter
root = Tk()
root.title("Reading PHSP")

# Thiết lập kích thước cố định cho cửa sổ
root.geometry("500x250")  # Đặt kích thước cửa sổ lớn hơn để hiển thị đầy đủ văn bản
root.resizable(False, False)  # Không cho phép thay đổi kích thước cửa sổ

# Thêm nút và nhãn vào giao diện với phông chữ lớn hơn và căn giữa
label = Label(root, text="Select Shase Space file .phsp", font=("Arial", 12), anchor='center', justify='center')
label.pack(pady=10)

button = Button(root, text="Select Shase Space file .phsp", command=analyze_files, font=("Arial", 12))
button.pack(pady=20)

# Chạy vòng lặp chính của Tkinter
root.mainloop()
